In [1]:
#Importing libraries

import psycopg2
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from distfit import distfit
import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Accessing the Primuss DB schema moodle and table quiz_attempts

conn = psycopg2.connect(dbname='primuss_fhin',
                            user='thisuccess',
                            password='5N7vcG!8FD%',
                            host='194.94.97.25',
                            port='8890')

cur = conn.cursor()
cur.execute("SELECT * FROM moodle.mdl21_quiz_attempts")

# Saving table into a list

records = cur.fetchall()

cur.close()
conn.close()

# Creating a data frae with our data

headers = ['id', 'uniqueid', 'quiz', 'userid', 'attempts', 'sumgrades', 'timestart',
           'timefinish', 'timemodified', 'layout', 'preview', 'currentpage', 'state',
           'timecheckstate', 'timemodifiedoffline']

#df = pd.read_csv('/Users/pagonis/Downloads/moodle_21_quiz_attempts.csv')
df = pd.DataFrame(records, columns = headers)



In [4]:
#Add from quiz table maxsumgrades, time_open, time_close, preferredbehaviour and grademethod
#only if quiz is identical to id from quiz table

maxsum = pd.read_excel('/home/pagonis/moodle_data/maxsumgrades_quiz.xlsx')

maxsum_dict = {maxsum['id'][i]:maxsum['sumgrades'][i] for i in range(len(maxsum))}
df['maxsumgrades'] = df['quiz'].replace(maxsum_dict)

maxsum_dict = {maxsum['id'][i]:maxsum['time_open'][i] for i in range(len(maxsum))}
df['time_open'] = df['quiz'].replace(maxsum_dict)

maxsum_dict = {maxsum['id'][i]:maxsum['time_close'][i] for i in range(len(maxsum))}
df['time_close'] = df['quiz'].replace(maxsum_dict)

maxsum_dict = {maxsum['id'][i]:maxsum['preferredbehaviour'][i] for i in range(len(maxsum))}
df['preferredbehaviour'] = df['quiz'].replace(maxsum_dict)

maxsum_dict = {maxsum['id'][i]:maxsum['grademethod'][i] for i in range(len(maxsum))}
df['grademethod'] = df['quiz'].replace(maxsum_dict)

maxsum_dict = {maxsum['id'][i]:maxsum['name'][i] for i in range(len(maxsum))}
df['name'] = df['quiz'].replace(maxsum_dict)

maxsum_dict = {maxsum['id'][i]:maxsum['course'][i] for i in range(len(maxsum))}
df['course'] = df['quiz'].replace(maxsum_dict)

#Transform columns for easier use

df['time_start'] = pd.to_datetime(df['timestart'], unit = 's')
df['time_finish'] = pd.to_datetime(df['timefinish'], unit = 's')
df['time_modified'] = pd.to_datetime(df['timemodified'], unit = 's')
df['sumgrades'] = pd.to_numeric(df['sumgrades'])
df['time_used'] = df['time_finish']-df['time_start']
df['time_used'] = df['timefinish']-df['timestart']
df['time_used'].where(df['time_used'] >= 0, None, inplace=True)
df['time_used'].where(df['time_used'] < 10000, None, inplace=True)

df = df.groupby("attempts").filter(lambda x: len(x) > 100)

time_used = pd.to_numeric(df['time_used'])
time_used = time_used.dropna()

#Creating grade column, here the sumgrades are normalised for all students in a scale from 0 to 100.
#Grades that are lower than 0 or higher that 100 after the tranformation are considered 
#equal to 0 and 100 ccordingly

df['grade'] = 100-((df['maxsumgrades']-df['sumgrades'])/df['maxsumgrades']*100)
df['grade'] = pd.to_numeric(df['grade'])
df['grade'].where(df['grade'] >= 0, 0, inplace=True)
df['grade'].where(df['grade'] < 100, 100, inplace=True)

In [5]:
df_trimmed = df[df['grade'].between(0.001, 99.999)]

In [6]:
def UserReindex(df = df, userid = 'userid'):
    if isinstance(df, pd.DataFrame):
        df_n = df.sort_values('userid', inplace=True)
        df_n = df.reset_index()
        user = df_n['userid'].to_list()
        from itertools import accumulate
        indexes  = range(len(user))
        byGroup  = accumulate(indexes,lambda i,u: (i+1)*(u>0 and user[u-1]==user[u]))
        indexes  = [i-1 for i in accumulate(int(g==0) for g in byGroup)]
        indexAndUser = [(i,u) for i,u in zip(indexes,user)]
        new_user = pd.DataFrame([(i,u) for i,u in zip(indexes,user)], columns=['new_user', 'old_user'])

        df_n['userid_n'] = new_user['new_user']
        df_n = df_n.drop('index', axis=1)
        return df_n
    else:
        raise TypeError('The imported object is not a pandas.DataFrame. Please import a pandas.DataFrame type.')

In [10]:
def StudentSimulator(df, size, add_method="reuse", userid = 'userid', seed=123):
    import random
    #Create the df
    data = pd.DataFrame(columns=["id", "userid", "quiz", "course", "time_used", "grade"])

    #Adding id
    data['id'] = list(range(1,size+1))
    
    np.random.seed(seed)
    vector = np.vectorize(np.int_)
    
    #Adding idx
    data['id'] = list(range(1,size+1))
    
    #Adding userid
    if add_method == 'reuse':
        #Creating the output dataframe
        newdf = pd.DataFrame()

        #Gather the unique users
        users = df[userid].unique()

        #Simulate students and reindexing
        ids = np.array(range(1, size+1))
        newuserid = []
        counter = 0

        while len(newdf) <= size:
            counter += 1
            user = int(np.random.choice(users, 1))
            t = df[userid]==user
            newdf = newdf.append(df[t]) 
            newuserid.append([counter]*t.sum())

        newuserid = [item for sublist in newuserid for item in sublist]
        newdf[userid] = newuserid

        newdf = newdf.reset_index()

        #Random remove the excess amount of observations 
        if len(newdf) > size:
            remove_n = len(newdf) - size
            drop_indices = np.random.choice(newdf.index, remove_n, replace=False)
            newdf1 = newdf.drop(drop_indices)
            
        users = newdf1['userid'].to_list()
        courses = newdf1['course'].to_list()
        items = newdf1['quiz'].to_list()
        data['userid'] = users
        data['course'] = courses
        data['quiz'] = items
       
    elif add_method=="simulate":
        #Adding userid accoring to statistics (mean=7, std=10)
        df = UserReindex(df = df, userid = userid)
        vector = np.vectorize(np.int_)
        uniques = int(size*df['userid_n'].nunique()/len(df))
        sampler = vector(np.linspace(start=1, stop=uniques, num=uniques))

        users = df['userid_n'].value_counts().rename_axis('userid').reset_index(name='counts')
        users_count = users['counts'].value_counts().rename_axis('no_of_grades').reset_index(name='no_of_students') 
        users_count = users_count.sort_values('no_of_grades')

        prob = stats.pareto.rvs(size=uniques, b=1.86, loc=-6.788, scale=7.788, random_state=seed)
        prob /= np.sum(prob)

        sampled_users = pd.Series(np.random.choice(sampler, size, p=prob))    
        data['userid'] = sampled_users


        #Adding courseid
        uniques = int(size*df['course'].nunique()/len(df))
        sampler = vector(np.linspace(start=1, stop=uniques, num=uniques))

        courses = df['course'].value_counts().rename_axis('courseid').reset_index(name='counts')
        courses_count = courses['counts'].value_counts().rename_axis('no_of_grades').reset_index(name='no_of_courses') 
        courses_count = courses_count.sort_values('no_of_grades')

        prob = stats.expon.rvs(size=uniques, loc=1, scale=58.928, random_state=seed)
        prob /= np.sum(prob)

        sampled_courses = pd.Series(np.random.choice(sampler, size, p=prob))    
        data['courseid'] = sampled_courses    

        #Adding items

        items = list(range(1, size+1))
        sampled_items = []
        data = data.sort_values('courseid')
        for i in tqdm(data['courseid'].unique(), desc='Finding quizzes'):
            myitems = [items.pop(random.randrange(len(items))) for _ in range(int(max(1, 0.15*len(data[data['courseid']==i]))))]
            sampled_items.append([random.choice(myitems) for _ in range(len(data[data['courseid']==i]))])


        data['quizid'] = sum(sampled_items, [])  

    #Adding grademethod
    diction = {df_trimmed['quiz'].iloc[i]:df_trimmed['grademethod'].iloc[i] for i in range(len(df_trimmed))}
    data['grademethod'] = data['quiz'].replace(diction)

    #Adding grades
    grade=stats.loggamma.rvs(c=0.07, loc=97.25, scale=1.84, size=size)
    data['grade'] = grade
    data['grade'].where(data['grade'] >= 0, 0, inplace=True)
    data['grade'].where(data['grade'] < 100, 100, inplace=True)

    #Adding time
    time=stats.lognorm.rvs(s=1.22, loc=-10.20, scale=389.70, size=size)
    data['time_used'] = time

    #for user in data['userid'].unique():
    #    for quiz in data['quiz'].unique():
    #        if data[data['userid']==user][data['quiz']==quiz]['quiz'].count()==1:
    #            data[data['userid']==user][data['quiz']==quiz]['attempts'] = 1
    #        else:
    #            data[data['userid']==user][data['quiz']==quiz]['attempts'] = 0

    return(data)

In [11]:
x = StudentSimulator(df_trimmed, 10000)

In [9]:
x.head()

,id,userid,quiz,attempts,grademethod,course,time_used,grade,courseid,quizid
6736,6737,723,NaN,NaN,NaN,NaN,159.661689,86.637762,1,649
5586,5587,1347,NaN,NaN,NaN,NaN,62.376220,33.226709,1,5554
2873,2874,638,NaN,NaN,NaN,NaN,567.699159,93.604754,1,5901
3528,3529,2354,NaN,NaN,NaN,NaN,442.692145,30.657571,1,5901
3138,3139,1280,NaN,NaN,NaN,NaN,117.911487,90.221025,1,3130
